In [34]:
from copy import copy
from Grammar import *

Search for productive symbols.

Let productive be the name of productive Vars.

Start with all symbols for each there is a Rule, which is a list of **only** Tokens,
and put such Vars to productive

On each step check for each rule that if Definition is a list of Tokens or Vars from result and add them to productive
until result is unchanged on a step

In [35]:
def remove_unproductive(grammar: Grammar) -> Grammar:

    # check if rule contain only tokens or vars from given set
    def contain_only_productive(rule: Rule, current_productive: set):
        for value in rule:
            if not (is_token(value) or value in current_productive):
                return False
        return True

    # check if there is a rule from Definition that contain_only_productive symbols
    def has_productive_rule(definition: Definition, current_productive: set) -> bool:
        for rule in definition:
            if contain_only_productive(rule, current_productive):
                return True
        return False

    # start is a set of productive symbols where rule contain only tokens
    productive = set(key for key, value in grammar.variables.items() if has_productive_rule(value, set()))
    prev_result = None

    while prev_result != productive:
        prev_result = productive.copy()
        productive = productive.union(
            set(var for var, definition in grammar.variables.items() if has_productive_rule(definition, productive)))

    result = copy(grammar)

    result.variables = {
        var: [rule for rule in definition
              if contain_only_productive(rule, productive)]  # remove rules containing unproductive symbols
        for var, definition in grammar.variables.items()
        if var in productive  # remove all vars that are unproductive
    }

    # if all variable are unproductive => remove main_var
    if not result.variables:
        result.main_var = ""

    return result

Search for accessible symbols.

Let accessible be the name of accessible Vars.

Start with main_var

On each step check for each rule for var from accessible and add all vars from these rules
until result is unchanged on a step

In [36]:
def remove_inaccessible(grammar: Grammar) -> Grammar:
    # check if rule contain specified var
    def rule_contains_inaccessible(rule: Rule, accessible_vars: set):
        for value in rule:
            if not (is_token(value) or value in accessible_vars):
                return False
        return True

    # get all accessible vars from definition
    def get_accessible_from_definition(definition: Definition) -> set:
        ret = set()
        for rule in definition:
            for value in rule:
                if is_var(value):
                    ret.add(value)
        return ret

    accessible = {grammar.main_var}
    prev_result = None

    while prev_result != accessible:
        prev_result = accessible.copy()
        for var, definition in grammar.variables.items():
            if var in accessible:
                accessible = accessible.union(get_accessible_from_definition(definition))

    result = copy(grammar)

    result.variables = {
        var: [rule for rule in definition
              if rule_contains_inaccessible(rule, accessible)]  # remove rules containing unproductive symbols
        for var, definition in grammar.variables.items()
        if var in accessible  # remove all vars that are unproductive
    }

    return result

In [37]:
def remove_exterior(grammar: Grammar) -> Grammar:
    return remove_inaccessible(remove_unproductive(grammar))

Search for vanish symbols.

Let vanish be the name of vanishing vars.

Start with all symbols for each there is a Rule which is a list of **only** 1 empty token,
and put such Vars to productive

On each step check for var if there is a path from it to only vanishing vars

In [38]:
def get_vanish_symbols(grammar: Grammar) -> set:
    # get all vanish symbols from definition
    def definition_contains_empty_rule(definition: Definition, vanish_symbols: set) -> bool:
        rule_empty = False
        for rule in definition:
            for value in rule:
                if is_token(value):
                    if not is_empty_token(value):
                        rule_empty = False
                        break
                else:  # is_var(value)
                    if value not in vanish_symbols:
                        rule_empty = False
                        break
                rule_empty = True

            if rule_empty:
               return True
        return False

    vanish = set(key for key, value in grammar.variables.items() if definition_contains_empty_rule(value, set()))
    prev_result = None

    while prev_result != vanish:
        prev_result = vanish.copy()
        vanish = vanish.union(
            key for key, value in grammar.variables.items() if definition_contains_empty_rule(value, vanish)
        )

    return vanish

In [39]:
def main():

    grammar = Grammar(
        {(1, "a"), (1, "b"), (0, "")},
        {
            "S": [["S", (0, ""), "T"], ["T"]],
            "T": [["S"], [(0, "")], ["R"]],
            "R": [["R"]],
            "F": [[(1, "a")]]
        },
        "S"
    )

    print(get_vanish_symbols(grammar), end="\n=======================\n")
    print(grammar, end="\n=======================\n")
    print(remove_exterior(grammar))

In [40]:
if __name__ == '__main__':
    main()


{'S', 'T'}
{(1, 'a'), (1, 'b'), (0, '')}

{'S': [['S', (0, ''), 'T'], ['T']], 'T': [['S'], [(0, '')], ['R']], 'R': [['R']], 'F': [[(1, 'a')]]}

S
{(1, 'a'), (1, 'b'), (0, '')}

{'S': [['S', (0, ''), 'T'], ['T']], 'T': [['S'], [(0, '')]]}

S
